In [1]:
import numpy as np
from aiida import load_profile
from matplotlib import gridspec, pyplot as plt
load_profile()
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, Data, StructureData, KpointsData, Dict, UpfData, Int, WorkChainNode, List, CifData, CalcFunctionNode
from aiida.plugins import CalculationFactory, DataFactory
from aiida.orm import QueryBuilder
from aiida.orm import load_node
import pandas as pd
import copy

In [2]:
cifs = pd.read_csv('cof-frameworks.csv')
cifs = cifs['CURATED-COFs ID'].values
print(cifs[:10])
print(len(cifs))

['05000N2' '05001N2' '07000N2' '07001N2' '07002N2' '07010N3' '07011N3'
 '07012N3' '07013N3' '08000N3']
626


# Gather H$_2$ Isotherm Data

In [3]:
h2_df = pd.DataFrame({'cof': [], 'h2_77K_1bar': [], 'h2_77K_5bar': [], 'h2_77K_100bar': [], 
                              'h2_198K_1bar': [], 'h2_198K_5bar': [], 'h2_198K_100bar': [], 
                              'h2_298K_1bar': [], 'h2_298K_5bar': [], 'h2_298K_100bar': [], 'ads_unit': []})

qb = QueryBuilder()
qb.append(Group, tag='group', project='uuid')
group_list = qb.all()

for group in group_list:
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'uuid': group[0], 'type_string': {'!==': 'core.import'}}, project='description')
    qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename')
    qb.append(Node, with_group='group', filters={'extras.tag4': 'isotmt_h2', 'attributes.is_porous': True}, project='attributes')
    res = qb.all()
    try:
        iso1 = np.zeros(3)
        iso5 = np.zeros(3)
        iso100 = np.zeros(3)
        for i in range(len(res[0][2]['isotherm'])):
            pressures = np.array(res[0][2]['isotherm'][i]['pressure'])
            idx1 = np.where(pressures == 1.0)[0][0]
            idx5 = np.where(pressures == 5.0)[0][0]
            idx100 = np.where(pressures == 100.0)[0][0]
            iso1[i] = res[0][2]['isotherm'][i]['loading_absolute_average'][idx1]
            iso5[i] = res[0][2]['isotherm'][i]['loading_absolute_average'][idx5]
            iso100[i] = res[0][2]['isotherm'][i]['loading_absolute_average'][idx100]
        temp_df = pd.DataFrame({"cof": [res[0][1]], "h2_77K_1bar": [iso1[0]], "h2_77K_5bar": [iso5[0]], "h2_77K_100bar": [iso100[0]], 
                         "h2_198K_1bar": [iso1[1]], "h2_198K_5bar": [iso5[1]], "h2_198K_100bar": [iso100[1]], 
                         "h2_298K_1bar": [iso1[2]], "h2_298K_5bar": [iso5[2]], "h2_298K_100bar": [iso100[2]], 'ads_unit': [res[0][2]['isotherm'][0]['loading_absolute_unit']]})
        h2_df = h2_df.append(temp_df, ignore_index=True)
    except:
        pass

In [4]:
h2_df

,cof,h2_77K_1bar,h2_77K_5bar,h2_77K_100bar,h2_198K_1bar,h2_198K_5bar,h2_198K_100bar,h2_298K_1bar,h2_298K_5bar,h2_298K_100bar,ads_unit
0,20040N2.cif,11.928642,21.431084,49.073250,0.278109,1.332727,15.775099,0.096364,0.478879,7.976579,mol/kg
1,19230N2.cif,8.840425,16.747312,40.077289,0.193945,0.929398,12.625859,0.077903,0.383508,6.602060,mol/kg
2,19030N2.cif,4.215130,7.705461,15.055241,0.079325,0.383566,4.787489,0.028881,0.143156,2.357657,mol/kg
3,20203N2.cif,10.261470,18.309024,43.259146,0.214183,1.034462,14.022267,0.088068,0.437082,7.365046,mol/kg
4,20117N3.cif,16.093796,40.296863,77.704705,0.365767,1.757583,23.698217,0.139735,0.703770,11.664606,mol/kg
5,15120N2.cif,8.774374,19.904065,64.401494,0.267202,1.310762,19.998174,0.128125,0.627980,11.391755,mol/kg
6,20182N2.cif,7.122019,16.293930,38.093787,0.165701,0.805212,11.446998,0.070288,0.347379,5.993868,mol/kg
7,20440N3.cif,11.062919,20.978152,47.135773,0.227411,1.128499,14.304101,0.086316,0.441635,7.110348,mol/kg
8,15010N2.cif,9.754361,18.444721,41.379890,0.207617,0.992490,13.080445,0.081284,0.404241,6.754088,mol/kg
9,20470N2.cif,9.386751,16.864616,37.119205,0.184160,0.889352,11.586259,0.070533,0.358674,5.839917,mol/kg


# Gather O$_2$ Isotherm Data

In [5]:
o2_df = pd.DataFrame({'cof': [], 'o2_1bar': [], 'o2_5bar': [], 'o2_10bar': [], 'o2_20bar': [],
                      'o2_30bar': [], 'o2_50bar': [], 'o2_80bar': [], 'o2_100bar': [],
                      'o2_140bar': [], 'o2_200bar': [], 'o2_ads_units': []})

qb = QueryBuilder()
qb.append(Group, tag='group', project='uuid')
group_list = qb.all()

for group in group_list:
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'uuid': group[0], 'type_string': {'!==': 'core.import'}}, project='description')
    qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename')
    qb.append(Node, with_group='group', filters={'extras.tag4': 'isot_o2', 'attributes.is_porous': True}, project='attributes')
    res = qb.all()
    try:
        temp_df = pd.DataFrame({'cof': [res[0][1]], 'o2_1bar': [res[0][2]['isotherm']['loading_absolute_average'][0]], 'o2_5bar': [res[0][2]['isotherm']['loading_absolute_average'][1]], 
                                'o2_10bar': [res[0][2]['isotherm']['loading_absolute_average'][2]], 'o2_20bar': [res[0][2]['isotherm']['loading_absolute_average'][3]],
                                'o2_30bar': [res[0][2]['isotherm']['loading_absolute_average'][4]], 'o2_50bar': [res[0][2]['isotherm']['loading_absolute_average'][5]], 
                                'o2_80bar': [res[0][2]['isotherm']['loading_absolute_average'][6]], 'o2_100bar': [res[0][2]['isotherm']['loading_absolute_average'][7]],
                                'o2_140bar': [res[0][2]['isotherm']['loading_absolute_average'][8]], 'o2_200bar': [res[0][2]['isotherm']['loading_absolute_average'][9]], 
                                'o2_ads_units': [res[0][2]['isotherm']['loading_absolute_unit']]})
        o2_df = o2_df.append(temp_df)
    except:
        pass

In [6]:
o2_df

,cof,o2_1bar,o2_5bar,o2_10bar,o2_20bar,o2_30bar,o2_50bar,o2_80bar,o2_100bar,o2_140bar,o2_200bar,o2_ads_units
0,20040N2.cif,0.491582,1.967337,3.571356,6.003287,8.153786,11.361495,14.951527,16.841481,19.912177,23.087333,mol/kg
0,19230N2.cif,0.276674,1.292307,2.429961,4.420706,6.025017,8.737986,11.850803,13.483395,16.139167,19.077228,mol/kg
0,19030N2.cif,0.130522,0.615209,1.127387,1.990159,2.663592,3.711884,4.827986,5.378791,6.192917,7.060610,mol/kg
0,20203N2.cif,0.311026,1.440141,2.767309,4.994912,6.835993,9.822118,13.143896,14.810566,17.700552,20.845967,mol/kg
0,20117N3.cif,0.516942,2.517011,4.867646,9.180739,12.890750,18.845302,25.061041,27.764070,31.953099,36.127919,mol/kg
0,15120N2.cif,0.315747,1.504277,2.843377,5.279188,7.699131,11.660998,16.713116,19.484873,24.357773,30.267835,mol/kg
0,20182N2.cif,0.218389,1.041523,2.022516,3.805232,5.371015,7.964767,10.910590,12.464960,14.918822,17.467671,mol/kg
0,20440N3.cif,0.389942,1.718736,3.058645,5.275983,7.005358,10.060351,13.346795,15.089111,17.780202,20.715341,mol/kg
0,15010N2.cif,0.315039,1.463245,2.691372,4.748823,6.515609,9.485940,12.652272,14.356294,17.019243,19.782153,mol/kg
0,20470N2.cif,0.294599,1.390486,2.477913,4.447969,5.875209,8.270309,10.906115,12.175158,14.362093,16.788363,mol/kg


# Gather CO$_2$ Isotherm Data

In [7]:
co2_df = pd.DataFrame({'cof': [], 'co2_0.001bar': [], 'co2_30bar': [], 'co2_henry': [], 'co2_ads_unit': [], 'co2_henry_unit': []})

qb = QueryBuilder()
qb.append(Group, tag='group', project='uuid')
group_list = qb.all()

for group in group_list:
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'uuid': group[0], 'type_string': {'!==': 'core.import'}}, project='description')
    qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename')
    qb.append(Node, with_group='group', filters={'extras.tag4': 'isot_co2'}, project='attributes')
    res = qb.all()
    try:
        temp_df = pd.DataFrame({'cof': [res[0][1]], 'co2_0.001bar': [res[0][2]['isotherm']['loading_absolute_average'][0]],
                                'co2_30bar': [res[0][2]['isotherm']['loading_absolute_average'][-1]], 'co2_henry': [res[0][2]['henry_coefficient_average']],
                                'co2_ads_unit': [res[0][2]['isotherm']['loading_absolute_unit']], 'co2_henry_unit': [res[0][2]['henry_coefficient_unit']]})
        co2_df = co2_df.append(temp_df)
    except:
        pass

In [8]:
co2_df

,cof,co2_0.001bar,co2_30bar,co2_henry,co2_ads_unit,co2_henry_unit
0,20040N2.cif,0.012433,28.009744,0.000124,mol/kg,mol/kg/Pa
0,19230N2.cif,0.001098,15.140952,0.000009,mol/kg,mol/kg/Pa
0,19030N2.cif,0.001720,7.810476,0.000021,mol/kg,mol/kg/Pa
0,20203N2.cif,0.002516,21.747351,0.000023,mol/kg,mol/kg/Pa
0,20117N3.cif,0.002481,41.325471,0.000023,mol/kg,mol/kg/Pa
0,15120N2.cif,0.000683,19.106967,0.000008,mol/kg,mol/kg/Pa
0,20182N2.cif,0.001172,21.430407,0.000012,mol/kg,mol/kg/Pa
0,20440N3.cif,0.009051,27.489843,0.000091,mol/kg,mol/kg/Pa
0,15010N2.cif,0.001038,17.679160,0.000011,mol/kg,mol/kg/Pa
0,20470N2.cif,0.002885,19.515196,0.000026,mol/kg,mol/kg/Pa


# Gather N$_2$ Isotherm Data

In [9]:
n2_df = pd.DataFrame({'cof': [], 'n2_0.001bar': [], 'n2_30bar': [], 'n2_henry': [], 'n2_ads_unit': [], 'n2_henry_unit': []})

qb = QueryBuilder()
qb.append(Group, tag='group', project='uuid')
group_list = qb.all()

for group in group_list:
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'uuid': group[0], 'type_string': {'!==': 'core.import'}}, project='description')
    qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename')
    qb.append(Node, with_group='group', filters={'extras.tag4': 'isot_n2'}, project='attributes')
    res = qb.all()
    try:
        temp_df = pd.DataFrame({'cof': [res[0][1]], 'n2_0.001bar': [res[0][2]['isotherm']['loading_absolute_average'][0]],
                                'n2_30bar': [res[0][2]['isotherm']['loading_absolute_average'][-1]], 'n2_henry': [res[0][2]['henry_coefficient_average']],
                                'n2_ads_unit': [res[0][2]['isotherm']['loading_absolute_unit']], 'n2_henry_unit': [res[0][2]['henry_coefficient_unit']]})
        n2_df = n2_df.append(temp_df)
    except:
        pass

In [10]:
n2_df

,cof,n2_0.001bar,n2_30bar,n2_henry,n2_ads_unit,n2_henry_unit
0,20040N2.cif,0.000400,6.697882,4.112790e-06,mol/kg,mol/kg/Pa
0,19230N2.cif,0.000197,4.376303,2.034090e-06,mol/kg,mol/kg/Pa
0,19030N2.cif,0.000088,1.920221,1.017770e-06,mol/kg,mol/kg/Pa
0,20203N2.cif,0.000325,5.957973,2.917640e-06,mol/kg,mol/kg/Pa
0,20117N3.cif,0.000731,10.858268,4.758380e-06,mol/kg,mol/kg/Pa
0,15120N2.cif,0.000240,5.955864,2.449330e-06,mol/kg,mol/kg/Pa
0,20182N2.cif,0.000237,4.763871,2.102950e-06,mol/kg,mol/kg/Pa
0,20440N3.cif,0.000370,6.169053,3.546910e-06,mol/kg,mol/kg/Pa
0,15010N2.cif,0.000324,4.682880,2.291180e-06,mol/kg,mol/kg/Pa
0,20470N2.cif,0.000237,5.187673,2.635640e-06,mol/kg,mol/kg/Pa


# Gather CH$_4$ Isotherm Data

In [11]:
ch4_df = pd.DataFrame({'cof': [], 'ch4_1bar': [], 'ch4_5.8bar': [], 'ch4_20bar': [], 'ch4_35bar': [], 'ch4_50bar': [], 'ch4_65bar': [], 'ch4_henry': [], 'ch4_ads_unit': [], 'ch4_henry_unit': []})

qb = QueryBuilder()
qb.append(Group, tag='group', project='uuid')
group_list = qb.all()

for group in group_list:
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'uuid': group[0], 'type_string': {'!==': 'core.import'}}, project='description')
    qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename')
    qb.append(Dict, with_group='group', filters={'extras.tag4': 'isot_ch4', 'attributes': {'has_key': 'isotherm'}}, project='attributes')
    res = qb.all()
    try:
        temp_df = pd.DataFrame({'cof': [res[0][1]], 'ch4_1bar': [res[0][2]['isotherm']['loading_absolute_average'][0]],
                                'ch4_5.8bar': [res[0][2]['isotherm']['loading_absolute_average'][1]], 'ch4_20bar': [res[0][2]['isotherm']['loading_absolute_average'][2]],
                                'ch4_35bar': [res[0][2]['isotherm']['loading_absolute_average'][3]], 'ch4_50bar': [res[0][2]['isotherm']['loading_absolute_average'][4]],
                                'ch4_65bar': [res[0][2]['isotherm']['loading_absolute_average'][5]], 'ch4_henry': [res[0][2]['henry_coefficient_average']],
                                'ch4_ads_unit': [res[0][2]['isotherm']['loading_absolute_unit']], 'ch4_henry_unit': [res[0][2]['henry_coefficient_unit']]})
        ch4_df = ch4_df.append(temp_df)
    except:
        pass

In [12]:
ch4_df

,cof,ch4_1bar,ch4_5.8bar,ch4_20bar,ch4_35bar,ch4_50bar,ch4_65bar,ch4_henry,ch4_ads_unit,ch4_henry_unit
0,20040N2.cif,1.114950,4.215052,9.718897,13.199184,15.661772,17.547083,0.000015,mol/kg,mol/kg/Pa
0,19230N2.cif,0.751647,3.420047,7.942129,10.919123,13.058423,14.637840,0.000008,mol/kg,mol/kg/Pa
0,19030N2.cif,0.396013,1.670599,3.605552,4.704974,5.398414,5.863654,0.000004,mol/kg,mol/kg/Pa
0,20203N2.cif,0.816947,3.833791,8.933379,11.886847,13.980822,15.631548,0.000008,mol/kg,mol/kg/Pa
0,20117N3.cif,1.161057,6.625412,19.032234,24.944133,28.001515,29.993279,0.000012,mol/kg,mol/kg/Pa
0,15120N2.cif,0.661847,3.274017,9.129162,13.555811,17.110591,19.964662,0.000007,mol/kg,mol/kg/Pa
0,20182N2.cif,0.520309,2.747868,7.399702,10.425317,12.476144,13.973889,0.000005,mol/kg,mol/kg/Pa
0,20440N3.cif,0.929735,3.679946,8.599292,11.949725,14.293030,15.903940,0.000011,mol/kg,mol/kg/Pa
0,15010N2.cif,0.884016,3.754153,8.847983,12.103984,14.245808,15.935515,0.000010,mol/kg,mol/kg/Pa
0,20470N2.cif,0.919154,3.848306,7.770515,10.132467,11.735586,13.093558,0.000009,mol/kg,mol/kg/Pa


# Gather Xe data

In [13]:
xe_df = pd.DataFrame({'cof': [], 'xe_henry': [], 'xe_henry_unit': []})

qb = QueryBuilder()
qb.append(Group, tag='group', project='uuid')
group_list = qb.all()

for group in group_list:
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'uuid': group[0], 'type_string': {'!==': 'core.import'}}, project='description')
    qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename')
    qb.append(Dict, with_group='group', filters={'extras.tag4': 'kh_xe'}, project='attributes')
    res = qb.all()
    try:
        temp_df = pd.DataFrame({'cof': [res[0][1]], 'xe_henry': [res[0][2]['henry_coefficient_average']], 'xe_henry_unit': [res[0][2]['henry_coefficient_unit']]})
        xe_df = xe_df.append(temp_df)
    except:
        pass

In [14]:
xe_df

,cof,xe_henry,xe_henry_unit
0,20040N2.cif,0.000101,mol/kg/Pa
0,19230N2.cif,0.000050,mol/kg/Pa
0,19030N2.cif,0.000034,mol/kg/Pa
0,20203N2.cif,0.000050,mol/kg/Pa
0,20117N3.cif,0.000044,mol/kg/Pa
0,15120N2.cif,0.000029,mol/kg/Pa
0,20182N2.cif,0.000023,mol/kg/Pa
0,20440N3.cif,0.000077,mol/kg/Pa
0,15010N2.cif,0.000067,mol/kg/Pa
0,20470N2.cif,0.000074,mol/kg/Pa


# Gather Kr data

In [15]:
kr_df = pd.DataFrame({'cof': [], 'kr_henry': [], 'kr_henry_unit': []})

qb = QueryBuilder()
qb.append(Group, tag='group', project='uuid')
group_list = qb.all()

for group in group_list:
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'uuid': group[0], 'type_string': {'!==': 'core.import'}}, project='description')
    qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename')
    qb.append(Dict, with_group='group', filters={'extras.tag4': 'kh_kr'}, project='attributes')
    res = qb.all()
    try:
        temp_df = pd.DataFrame({'cof': [res[0][1]], 'kr_henry': [res[0][2]['henry_coefficient_average']], 'kr_henry_unit': [res[0][2]['henry_coefficient_unit']]})
        kr_df = kr_df.append(temp_df)
    except:
        pass

In [16]:
kr_df

,cof,kr_henry,kr_henry_unit
0,20040N2.cif,0.000020,mol/kg/Pa
0,19230N2.cif,0.000009,mol/kg/Pa
0,19030N2.cif,0.000005,mol/kg/Pa
0,20203N2.cif,0.000010,mol/kg/Pa
0,20117N3.cif,0.000013,mol/kg/Pa
0,15120N2.cif,0.000007,mol/kg/Pa
0,20182N2.cif,0.000006,mol/kg/Pa
0,20440N3.cif,0.000013,mol/kg/Pa
0,15010N2.cif,0.000011,mol/kg/Pa
0,20470N2.cif,0.000011,mol/kg/Pa


# Gather H$_2$O data

In [17]:
h2o_df = pd.DataFrame({'cof': [], 'h2o_henry': [], 'h2o_henry_unit': []})

qb = QueryBuilder()
qb.append(Group, tag='group', project='uuid')
group_list = qb.all()

for group in group_list:
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'uuid': group[0], 'type_string': {'!==': 'core.import'}}, project='description')
    qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename')
    qb.append(Dict, with_group='group', filters={'extras.tag4': 'kh_h2o'}, project='attributes')
    res = qb.all()
    try:
        temp_df = pd.DataFrame({'cof': [res[0][1]], 'h2o_henry': [res[0][2]['henry_coefficient_average']], 'h2o_henry_unit': [res[0][2]['henry_coefficient_unit']]})
        h2o_df = h2o_df.append(temp_df)
    except:
        pass

In [18]:
h2o_df

,cof,h2o_henry,h2o_henry_unit
0,20040N2.cif,7.956750e-05,mol/kg/Pa
0,19230N2.cif,5.414730e-06,mol/kg/Pa
0,19030N2.cif,8.276700e-04,mol/kg/Pa
0,20203N2.cif,8.476640e-05,mol/kg/Pa
0,20117N3.cif,1.370200e-05,mol/kg/Pa
0,15120N2.cif,3.877280e-06,mol/kg/Pa
0,20182N2.cif,1.676440e-05,mol/kg/Pa
0,20440N3.cif,1.496950e-03,mol/kg/Pa
0,15010N2.cif,5.387700e-06,mol/kg/Pa
0,20470N2.cif,1.192400e-04,mol/kg/Pa


# Gather H$_2$S data

In [19]:
h2s_df = pd.DataFrame({'cof': [], 'h2s_henry': [], 'h2s_henry_unit': []})

qb = QueryBuilder()
qb.append(Group, tag='group', project='uuid')
group_list = qb.all()

for group in group_list:
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'uuid': group[0], 'type_string': {'!==': 'core.import'}}, project='description')
    qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename')
    qb.append(Dict, with_group='group', filters={'extras.tag4': 'kh_h2s'}, project='attributes')
    res = qb.all()
    try:
        temp_df = pd.DataFrame({'cof': [res[0][1]], 'h2s_henry': [res[0][2]['henry_coefficient_average']], 'h2s_henry_unit': [res[0][2]['henry_coefficient_unit']]})
        h2s_df = h2s_df.append(temp_df)
    except:
        pass

In [20]:
h2s_df

,cof,h2s_henry,h2s_henry_unit
0,20040N2.cif,0.000312,mol/kg/Pa
0,19230N2.cif,0.000043,mol/kg/Pa
0,19030N2.cif,0.000086,mol/kg/Pa
0,20203N2.cif,0.000063,mol/kg/Pa
0,20117N3.cif,0.000046,mol/kg/Pa
0,15120N2.cif,0.000029,mol/kg/Pa
0,20182N2.cif,0.000024,mol/kg/Pa
0,20440N3.cif,0.000205,mol/kg/Pa
0,15010N2.cif,0.000060,mol/kg/Pa
0,20470N2.cif,0.000105,mol/kg/Pa


# Combine DataFrames

In [21]:
df = copy.deepcopy(o2_df)
df = df.merge(co2_df, on='cof', how='outer')
df = df.merge(n2_df, on='cof', how='outer')
df = df.merge(h2_df, on='cof', how='outer')
df = df.merge(h2o_df, on='cof', how='outer')
df = df.merge(h2s_df, on='cof', how='outer')
df = df.merge(xe_df, on='cof', how='outer')
df = df.merge(kr_df, on='cof', how='outer')
df = df.merge(ch4_df, on='cof', how='outer')

In [22]:
df

,cof,o2_1bar,o2_5bar,o2_10bar,o2_20bar,o2_30bar,o2_50bar,o2_80bar,o2_100bar,o2_140bar,...,kr_henry_unit,ch4_1bar,ch4_5.8bar,ch4_20bar,ch4_35bar,ch4_50bar,ch4_65bar,ch4_henry,ch4_ads_unit,ch4_henry_unit
0,20040N2.cif,0.491582,1.967337,3.571356,6.003287,8.153786,11.361495,14.951527,16.841481,19.912177,...,mol/kg/Pa,1.114950,4.215052,9.718897,13.199184,15.661772,17.547083,0.000015,mol/kg,mol/kg/Pa
1,19230N2.cif,0.276674,1.292307,2.429961,4.420706,6.025017,8.737986,11.850803,13.483395,16.139167,...,mol/kg/Pa,0.751647,3.420047,7.942129,10.919123,13.058423,14.637840,0.000008,mol/kg,mol/kg/Pa
2,19030N2.cif,0.130522,0.615209,1.127387,1.990159,2.663592,3.711884,4.827986,5.378791,6.192917,...,mol/kg/Pa,0.396013,1.670599,3.605552,4.704974,5.398414,5.863654,0.000004,mol/kg,mol/kg/Pa
3,20203N2.cif,0.311026,1.440141,2.767309,4.994912,6.835993,9.822118,13.143896,14.810566,17.700552,...,mol/kg/Pa,0.816947,3.833791,8.933379,11.886847,13.980822,15.631548,0.000008,mol/kg,mol/kg/Pa
4,20117N3.cif,0.516942,2.517011,4.867646,9.180739,12.890750,18.845302,25.061041,27.764070,31.953099,...,mol/kg/Pa,1.161057,6.625412,19.032234,24.944133,28.001515,29.993279,0.000012,mol/kg,mol/kg/Pa
5,15120N2.cif,0.315747,1.504277,2.843377,5.279188,7.699131,11.660998,16.713116,19.484873,24.357773,...,mol/kg/Pa,0.661847,3.274017,9.129162,13.555811,17.110591,19.964662,0.000007,mol/kg,mol/kg/Pa
6,20182N2.cif,0.218389,1.041523,2.022516,3.805232,5.371015,7.964767,10.910590,12.464960,14.918822,...,mol/kg/Pa,0.520309,2.747868,7.399702,10.425317,12.476144,13.973889,0.000005,mol/kg,mol/kg/Pa
7,20440N3.cif,0.389942,1.718736,3.058645,5.275983,7.005358,10.060351,13.346795,15.089111,17.780202,...,mol/kg/Pa,0.929735,3.679946,8.599292,11.949725,14.293030,15.903940,0.000011,mol/kg,mol/kg/Pa
8,15010N2.cif,0.315039,1.463245,2.691372,4.748823,6.515609,9.485940,12.652272,14.356294,17.019243,...,mol/kg/Pa,0.884016,3.754153,8.847983,12.103984,14.245808,15.935515,0.000010,mol/kg,mol/kg/Pa
9,20470N2.cif,0.294599,1.390486,2.477913,4.447969,5.875209,8.270309,10.906115,12.175158,14.362093,...,mol/kg/Pa,0.919154,3.848306,7.770515,10.132467,11.735586,13.093558,0.000009,mol/kg,mol/kg/Pa


In [24]:
df.dropna().to_csv('aiida_ads_data_oct20.csv', index=False)

# Application DataFrame

In [51]:
appl_df = pd.DataFrame({'cof': [], 'ch4storage_298K': [], 'h2sh2osel_300K': [],
                        'h2storage_a': [], 'h2storage_b': [], 'h2storage_c': [],
                        'o2storage_298K': [], 'xekrsel_300K': []})

qb = QueryBuilder()
qb.append(Group, tag='group', project='uuid')
group_list = qb.all()

for (i, group) in enumerate(group_list):
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'uuid': group[0], 'type_string': {'!==': 'core.import'}})
    qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename') # [0]
    qb.append(Dict, with_group='group', filters={'extras.tag4': 'appl_h2sh2osel'}, project='attributes.selectivity_average') # [1]
    qb.append(Dict, with_group='group', filters={'extras.tag4': 'appl_xekrsel'}, project='attributes.selectivity_average') # [2]
    qb.append(Dict, with_group='group', filters={'extras.tag4': 'appl_ch4storage'}, project='attributes.wc_65bar_mol/kg_average') # [3]
    qb.append(Dict, with_group='group', filters={'extras.tag4': 'appl_o2storage'}, project='attributes.wc_140bar_mol/kg_average') # [4]
    qb.append(Dict, with_group='group', filters={'extras.tag4': 'appl_h2storage'}, project='attributes') # [5, 6, 7]
    res = qb.all()
    if i % 25 == 0:
        print(res)
    try:
        temp_df = pd.DataFrame({'cof': [res[0][0]], 'h2sh2osel_300K': [res[0][1]], 'xekrsel_300K': [res[0][2]],
                                'ch4storage_298K': [res[0][3]], 'o2storage_298K': [res[0][4]], 
                                'h2storage_a': [res[0][5]['case-a_g/L_average']], 'h2storage_b': [res[0][5]['case-b_g/L_average']],
                                'h2storage_c': [res[0][5]['case-c_g/L_average']]})
        appl_df = appl_df.append(temp_df)
    except:
        pass

[['20040N2.cif', 3.9242906965784, 5.1097575107188, 13.332031016555, 17.94484064308, {'case-a_g/L_dev': 0.089366252135032, 'case-a_wt%_dev': 0.019243461887224, 'case-b_g/L_dev': 0.24935180585974, 'case-b_wt%_dev': 0.05369355722026, 'case-c_g/L_dev': 0.23073825113023, 'case-c_wt%_dev': 0.049685453238436, 'case-a_g/L_unit': 'g/L', 'case-a_wt%_unit': 'wt%', 'case-b_g/L_unit': 'g/L', 'case-b_wt%_unit': 'wt%', 'case-c_g/L_unit': 'g/L', 'case-c_wt%_unit': 'wt%', 'case-a_g/L_average': 14.207068283932, 'case-a_wt%_average': 3.0592440716653, 'case-b_g/L_average': 25.673933052074, 'case-b_wt%_average': 5.5284331655334, 'case-c_g/L_average': 34.499762928472, 'case-c_wt%_average': 7.4289215131142}]]
[['19022N3.cif', 1.9751783876695, 3.3140662167316, 22.297246059984, 27.120949664886, {'case-a_g/L_dev': 0.13731743658352, 'case-a_wt%_dev': 0.037361527297528, 'case-b_g/L_dev': 0.11684927886855, 'case-b_wt%_dev': 0.03179252126141, 'case-c_g/L_dev': 0.12077098562649, 'case-c_wt%_dev': 0.032859544923772, 

In [52]:
appl_df

,ch4storage_298K,cof,h2sh2osel_300K,h2storage_a,h2storage_b,h2storage_c,o2storage_298K,xekrsel_300K
0,13.332,20040N2.cif,3.92429,14.207068,25.673933,34.499763,17.944841,5.10976
0,11.2178,19230N2.cif,7.97395,12.534526,23.886770,31.982364,14.846860,5.6092
0,4.19306,19030N2.cif,0.104393,9.202310,14.562898,21.478660,5.577708,6.59827
0,11.7978,20203N2.cif,0.740857,12.690710,23.307356,30.825043,16.260411,5.27401
0,23.3679,20117N3.cif,3.38874,17.018007,27.685245,45.597740,29.436087,3.53846
0,16.6906,15120N2.cif,7.50369,12.463441,28.631159,35.792380,22.853496,3.96722
0,11.226,20182N2.cif,1.45047,12.030211,23.627557,33.568441,13.877299,3.92533
0,12.224,20440N3.cif,0.137203,13.721984,25.892541,35.707293,16.061466,5.95118
0,12.1814,15010N2.cif,11.101,13.051801,23.614738,32.562593,15.555998,6.29116
0,9.24525,20470N2.cif,0.879931,12.346975,22.273972,30.497380,12.971607,6.88645


In [54]:
appl_df.dropna().to_csv("appl_aiida_data.csv")

In [41]:
tag4s = []
for group in group_list:
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'uuid': group[0]})
    qb.append(Dict, with_group='group')
    qb.limit(100)
    res = qb.all()
    for r in res:
        try:
            tag4s.append(r[0].extras['tag4'])
        except:
            pass
unique_tag4s = np.unique(tag4s)
print(unique_tag4s)

['appl_ch4storage' 'appl_h2sh2osel' 'appl_h2storage' 'appl_o2storage'
 'appl_pecoal' 'appl_peng' 'appl_xekrsel' 'dftopt' 'isot_ch4' 'isot_co2'
 'isot_n2' 'isot_o2' 'isotmt_h2' 'kh_h2o' 'kh_h2s' 'kh_kr' 'kh_xe'
 'opt_zeopp' 'orig_zeopp']


In [ ]:
appl_df

In [6]:
qb = QueryBuilder()
qb.append(Group, tag='group', project='uuid')
group_list = qb.all()

for group in group_list:
    qb = QueryBuilder()
    qb.append(Group, tag='group', filters={'uuid': group[0], 'type_string': {'!==': 'core.import'}}, project='description')
    qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename')
    qb.append(Dict, with_group='group', filters={'extras.tag4': 'orig_zeopp'}, project='attributes')
    qb.append(Dict, with_group='group', filters={'extras.tag4': 'opt_zeopp'}, project='attributes')
    res = qb.all()
    try:
        name = res[0][1]
        orig_dict = res[0][2]
        opt_dict = res[0][3]
#        print("zeopp_results/orig" + name.split('.')[0] + '.json')
        with open("zeopp_results/orig" + name.split('.')[0] + '.json', 'w') as outfile:
            json.dump(orig_dict, outfile)
        with open("zeopp_results/opt" + name.split('.')[0] + '.json', 'w') as outfile:
            json.dump(opt_dict, outfile)
    except:
        pass

In [4]:
qb = QueryBuilder()
qb.append(Group, tag='group', project='uuid', filters={'description': {'!==': ''}})
group_list = qb.all()
qb = QueryBuilder()
qb.append(Group, tag='group', filters={'uuid': group_list[0][0]}, project='description')
qb.append(CifData, with_group='group', filters={'extras.tag4': 'orig_cif'}, project='attributes.filename')
qb.append(Node, with_group='group', filters={'extras.tag4': 'orig_zeopp'}, project='*')
res = qb.all()

IndexError: list index out of range

res

In [5]:
import json

In [ ]:
res[0][1].attributes
with open("sample.json", "w") as outfile:  
    json.dump(res[0][1].attributes, outfile) 
